<a href="https://colab.research.google.com/github/vedsule/noyo-san-t/blob/main/Research/tv3_noyo_san(Emo_%2B_Senti_Roberta_base).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *Emotion Recognition*

## *Model : twitter-roberta-base-emotion*

*4 categories ['anger', 'joy', 'optimism', 'sadness']*

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

In [ ]:
task='emotion'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)

config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

### *lable mapping*

In [ ]:
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]
labels

['anger', 'joy', 'optimism', 'sadness']

### *for PyTorch*

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
text = "Good night 😊"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

In [ ]:
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) joy 0.4426
2) sadness 0.2444
3) anger 0.1741
4) optimism 0.1389


### *for TensorFlow*

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
text = "Good night 😊"
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)
scores = output[0][0].numpy()
scores = softmax(scores)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-emotion.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [ ]:
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) joy 0.4426
2) sadness 0.2444
3) anger 0.1741
4) optimism 0.1389


In [ ]:
def get_emo_score(text):
  encoded_input = tokenizer(text, return_tensors='tf')
  output = model(encoded_input)
  scores = output[0][0].numpy()
  scores = softmax(scores)

  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  for i in range(scores.shape[0]):
      l = labels[ranking[i]]
      s = scores[ranking[i]]
      print(f"{i+1}) {l} {np.round(float(s), 4)}")

In [ ]:
get_emo_score("fuck you bitch")

1) joy 0.3183
2) sadness 0.2686
3) anger 0.2234
4) optimism 0.1896


## *Model : mrm8488/t5-base-finetuned-emotion*

In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead

MODEL2 = "mrm8488/t5-base-finetuned-emotion"

In [ ]:
tokenizer2 = AutoTokenizer.from_pretrained(MODEL2)

In [ ]:
model2 = AutoModelWithLMHead.from_pretrained(MODEL2)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1699: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
text = "i feel as if i havent blogged in ages are at least truly blogged i am doing an update cute"
encoded_text = tokenizer2.encode(text , return_tensors='pt')
output = model2.generate(input_ids = encoded_text, max_length=2)

In [ ]:
decode_output =  [tokenizer2.decode(ids) for ids in output]
label = decode_output[0]
label

'<pad> joy'

In [ ]:
get_emo("i am going to kill you")

'<pad> anger'

# *Sentiment Detetion*

## *Model : twitter-roberta-base-sentiment*

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [ ]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
example = "i love this cookies"

tokenizer (example, return_tensors='pt')
encoded_text = tokenizer (example, return_tensors='pt')
output = model(**encoded_text)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

In [ ]:
scores_dict = {
    'roberta_neg' : scores[0],
    'roberta_neu' : scores[1],
    'roberta_pos' : scores[2]
}
print(scores_dict)

{'roberta_neg': 0.0041405065, 'roberta_neu': 0.015297282, 'roberta_pos': 0.98056227}
